# Music VAD Model

In [1]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

import tensorflow as tf
# import tensorflow_datasets as tfds

# import adjustText
import itertools
# import os
# import csv
# import collections
# import re

# from tqdm.auto import tqdm
save_figures = False
data_path = "../Data"
figure_path = "../Figures"

## Load preprocessed data

In [2]:
mood_dict = dict(
    zip(range(276,283),
        ['Happy music', 'Funny music', 'Sad music', 'Tender music', 
         'Exciting music', 'Angry music', 'Scary music']
    )
)
print(mood_dict)

{276: 'Happy music', 277: 'Funny music', 278: 'Sad music', 279: 'Tender music', 280: 'Exciting music', 281: 'Angry music', 282: 'Scary music'}


In [3]:
bal_train_context_pd = pd.read_parquet(f"{data_path}/bal_train_music_contexts_full.parquet")
bal_train_embeddings = np.load(f"{data_path}/bal_train_music_embeddings.npy")

eval_context_pd = pd.read_parquet(f"{data_path}/eval_music_contexts_full.parquet")
eval_embeddings = np.load(f"{data_path}/eval_music_embeddings.npy")

unbal_train_context_pd = pd.read_parquet(f"{data_path}/unbal_train_music_contexts_full.parquet")
unbal_train_embeddings = np.load(f"{data_path}/unbal_train_music_embeddings.npy")

In [4]:
print(bal_train_embeddings.shape)
print(eval_embeddings.shape)
print(unbal_train_embeddings.shape)

display(bal_train_context_pd.head())
display(bal_train_context_pd.tail())

display(eval_context_pd.head())
display(eval_context_pd.tail())

display(unbal_train_context_pd.head())
display(unbal_train_context_pd.tail())

(421, 10, 128)
(420, 10, 128)
(16114, 10, 128)


,youtube_id,start_time,end_time,mood,valence,arousal,dominance,length,Weight
0,b'0khKvVDyYV4',240.0,250.0,276,1.0,0.735,0.772,10.0,0.170389
1,b'8Yt55huZGZc',30.0,40.0,276,1.0,0.735,0.772,10.0,0.170389
2,b'RSmNh8gfqg8',240.0,250.0,276,1.0,0.735,0.772,10.0,0.170389
3,b'1-dMI6S5rLs',21.0,31.0,276,1.0,0.735,0.772,10.0,0.170389
4,b'-mA_bqD1tgU',30.0,40.0,276,1.0,0.735,0.772,10.0,0.170389


,youtube_id,start_time,end_time,mood,valence,arousal,dominance,length,Weight
416,b'N0i99VyZCg8',70.0,80.0,282,0.062,0.952,0.528,10.0,0.14099
417,b'YEwikeeqF38',330.0,340.0,282,0.062,0.952,0.528,10.0,0.14099
418,b'hNKTrKk4hZs',80.0,90.0,282,0.062,0.952,0.528,10.0,0.14099
419,b'Hdko95EitpI',40.0,50.0,282,0.062,0.952,0.528,10.0,0.14099
420,b'A-7dmBdsFXc',70.0,80.0,282,0.062,0.952,0.528,10.0,0.14099


,youtube_id,start_time,end_time,mood,valence,arousal,dominance,Weight
0,b'xFHF_1FhtIo',110.0,120.0,276,1.0,0.735,0.772,0.170389
1,b'0KCVgexi4yU',30.0,40.0,276,1.0,0.735,0.772,0.170389
2,b'euAQCWBX6ns',40.0,50.0,276,1.0,0.735,0.772,0.170389
3,b'2M9GSksX_ho',30.0,40.0,276,1.0,0.735,0.772,0.170389
4,b'rOn6IPrdXUo',80.0,90.0,276,1.0,0.735,0.772,0.170389


,youtube_id,start_time,end_time,mood,valence,arousal,dominance,Weight
415,b'Qa-Qs9CtOOw',30.0,40.0,282,0.062,0.952,0.528,0.14099
416,b'iFOwlZfNTO4',30.0,40.0,282,0.062,0.952,0.528,0.14099
417,b'n7WM-2sI2Uo',410.0,420.0,282,0.062,0.952,0.528,0.14099
418,b'VeuetUsa9d0',30.0,40.0,282,0.062,0.952,0.528,0.14099
419,b'k-J2-Ou1Fm8',30.0,40.0,282,0.062,0.952,0.528,0.14099


,youtube_id,start_time,end_time,mood,valence,arousal,dominance,length,Weight
0,b'8FPA_Y7sssI',30.0,40.0,276,1.0,0.735,0.772,10.0,0.170389
1,b'8ffdVTnJdKI',30.0,40.0,276,1.0,0.735,0.772,10.0,0.170389
2,b'AeY-sUZDSKY',230.0,240.0,276,1.0,0.735,0.772,10.0,0.170389
3,b'Bj7u7piJfXc',24.0,34.0,276,1.0,0.735,0.772,10.0,0.170389
4,b'bj8XI04XnNQ',30.0,40.0,276,1.0,0.735,0.772,10.0,0.170389


,youtube_id,start_time,end_time,mood,valence,arousal,dominance,length,Weight
16109,b'F5mgSqN9hKA',0.0,10.0,282,0.062,0.952,0.528,10.0,0.14099
16110,b'53Dqp73SGJY',170.0,180.0,282,0.062,0.952,0.528,10.0,0.14099
16111,b'p0AfKyseeNg',70.0,80.0,282,0.062,0.952,0.528,10.0,0.14099
16112,b'i7q_PlWGJUM',10.0,20.0,282,0.062,0.952,0.528,10.0,0.14099
16113,b'_YPRM2_IgOY',110.0,120.0,282,0.062,0.952,0.528,10.0,0.14099


### VAD mapping

In [5]:
music_VAD_mapping = np.array([
    [1,0.735,0.772],
    [0.918,0.61,0.566],
    [0.225,0.333,0.149],
    [0.63,0.52,0.509],
    [0.95,0.792,0.789],
    [0.122,0.83,0.604],
    [0.062,0.952,0.528],
])

In [6]:
music_vad_mapping_pd = pd.DataFrame(
    columns = ["valence", "arousal", "dominance"], 
    data = music_VAD_mapping, 
    index = ['Happy music', 'Funny music', 'Sad music', 'Tender music', 
             'Exciting music', 'Angry music', 'Scary music'])

display(music_vad_mapping_pd)

,valence,arousal,dominance
Happy music,1.000,0.735,0.772
Funny music,0.918,0.610,0.566
Sad music,0.225,0.333,0.149
Tender music,0.630,0.520,0.509
Exciting music,0.950,0.792,0.789
Angry music,0.122,0.830,0.604
Scary music,0.062,0.952,0.528


## Model

In [7]:
class Music_VAD(tf.keras.Model):
    def __init__(self):
        super(Music_VAD, self).__init__()
        self.music_length = 10 # 10 seconds
        self.VGG_size = 128 # VGG embedding size

        # TODO:
        # 1) Define any hyperparameters

        # Define batch size and optimizer/learning rate
        self.batch_size = 128 # You probably should change this
        self.optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01) ## what should the learning rate be????


        # 2) Define embeddings, encoder, decoder, and feed forward layer
        
        # Convolution? LSTM? Bi-directional LSTM Dense?
        # How many layers?
        self.next_layers_1 = None 
        self.next_layers_2 = None 
        self.next_layers_3 = None 

    @tf.function
    def call(self, VGGish_input):
        """
        :param VGGish_input: batched input of the VGGish embeddings, [batch_size x 10 x 128 x 1]
        :return prbs: The 3d VAD coordinates as a tensor, [batch_size x 3]
        """

        # TODO:
        # 1) Pass your French sentence embeddings to your encoder
        # 2) Pass your English sentence embeddings, and final state of your encoder, to your decoder
        # 3) Apply dense layer(s) to the decoder out to generate probabilities
        
        
        ## do something here
        
        VAD_coordinates = None
        
        # The final result should look like this
        # VAD_coordinates = 
        #     [[v_1, a_1, d_1],
        #      [v_2, a_2, d_2],
        #      ...
        #      [v_batch, a_batch, d_batch]]
        
        return VAD_coordinates


    def loss_function(self, VAD_true, VAD_pred, sample_weights):
        """
        Calculates the Euclidean distance between the true VAD coordinates and the predicted coordinates.

        :param VAD_true: float tensor, [batch_size x 3]
        :param VAD_pred: float tensor, [batch_size x 3]
        :param sample_weights: float tensor, [batch_size x 1]

        :return: the loss of the model as a tensor
        """
        
        ## Return the weighted average of euclidean distances
        ## loss = (weight_1*distance_1 + weight_1*distance_2 + ... weight_n*distance_n)/(n_batch)

        return None

## Customized train, test, accuracy functions

In [8]:
def train(model, VGGish_input, VAD_true, sample_weights):
    """
    Runs through one epoch - all training examples.

    :param model: the initialized model to use for forward and backward pass
    :param VGGish_input: batched input of the VGGish embeddings, [batch_size x 10 x 128 x 1]
    :param VAD_true: float tensor, [batch_size x 3]
    :param sample_weights: float tensor, [batch_size x 1]
    
    :return: None
    """

    input_size = len(text_input)
    batch_size = model.batch_size
    
    # drop the rest of the data that do not fit in batches
    input_size = input_size - (input_size%batch_size)
    
    # shuffle
    shuffled_index = tf.random.shuffle(tf.range(input_size))
    shuffled_text_input  = tf.gather(text_input, shuffled_index)
    shuffled_VAD_true = tf.gather(VAD_true, shuffled_index)
    
    for train_index in range(0, input_size, batch_size): 
        batch_text_input = shuffled_text_input[train_index:(train_index + batch_size)]
        batch_VAD_true   = shuffled_VAD_true[train_index:(train_index + batch_size)]

        with tf.GradientTape() as tape:
            # get the loss for this batch
            batch_VAD_pred = model(batch_text_input, 
                                   batch_VAD_true)
            batch_loss = model.loss_function(batch_VAD_true, 
                                             batch_VAD_pred, 
                                             sample_weights)
            
        gradients = tape.gradient(batch_loss, model.trainable_variables)
        
        model.optimizer.apply_gradients(zip(gradients, trainable_vars))
        
        if train_index%(batch_size*200) == 0:
            print(f"training on batch {train_index}, "
                  f"progress {100*(train_index + batch_size)/(input_size):2.1f}%, "
                  f"loss {batch_loss:.4f}")
    
    print(f"training over\n"
          f"last batch {train_index}, "
          f"progress {100*(train_index + batch_size)/(input_size):2.1f}%, "
          f"loss {batch_loss:.4f}")    

In [9]:
def test(model, VGGish_input, VAD_true):
    """
    Runs through one epoch - all training examples.

    :param model: the initialized model to use for forward and backward pass
    :param VGGish_input: batched input of the VGGish embeddings, [batch_size x 10 x 128 x 1]
    :param VAD_true: float tensor, [batch_size x 3]
    
    :return: Loss
    """
    
    ## Do something similar to train

    return None

In [10]:
def accuracy(model, VGGish_input, emotion_labels, music_vad_mapping_pd):
    """
    Runs through one epoch - all training examples.

    Predict the VAD coordinate for the input music
    Find the closest emotion label on the VAD space
    See if the model has chosen the correct emotion label

    :param model: the initialized model to use for forward and backward pass
    :param VGGish_input: batched input of the VGGish embeddings, [batch_size x 10 x 128 x 1]
    :param emotion_labels: boolean tensor, [batch_size, ]
    
    :return: average accuracy
    """
    
    ## return the average accuracy

    return None

## Train/test the model

### Prepare the input tensors

In [11]:
VGGish_input_train = unbal_train_embeddings.reshape(-1, 10, 128, 1)
VGGish_input_test = eval_embeddings.reshape(-1, 10, 128, 1)

VAD_true_train = unbal_train_context_pd[["valence", "arousal", "dominance"]].to_numpy()
VAD_true_test = eval_context_pd[["valence", "arousal", "dominance"]].to_numpy()

In [12]:
print(VGGish_input_train.shape)
print(VGGish_input_test.shape)

print(VAD_true_train.shape)
print(VAD_true_test.shape)

(16114, 10, 128, 1)
(420, 10, 128, 1)
(16114, 3)
(420, 3)


In [13]:
sample_weights_train = unbal_train_context_pd["Weight"].to_numpy()
print(sample_weights_train.shape)

sample_weights_test = eval_context_pd["Weight"].to_numpy()
print(sample_weights_test.shape)

(16114,)
(420,)


In [14]:
emotion_labels_train = unbal_train_context_pd["mood"].to_numpy()
print(emotion_labels_train.shape)
print("")

emotion_labels_test = eval_context_pd["mood"].to_numpy()
print(emotion_labels_test.shape)

(16114,)

(420,)


### Initialize the model and train

In [15]:
n_epochs = 3 ## how many epochs are needed???

model = Music_VAD()

# for each_epoch in range(n_epochs):
#     train(model, VGGish_input, VAD_true, sample_weights)

In [16]:
# test_loss = test(model, VGGish_input_test, VAD_true_test)
# test_accuracy = accuracy(model, VGGish_input_test, emotion_labels_test, music_vad_mapping_pd)
# print(f"test loss: {test_loss}")
# print(f"test accuracy: {100*test_accuracy}%")